# Music Recommendation -- Random Forest, XGboost

In [ ]:
# Load Python libraries
from sklearn import cross_validation, grid_search, metrics, ensemble
import xgboost as xgb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

## Data Preparation

I merged 3 datasets (train, songs, members), replaced NAs, created 6 columns (days, months, years of registration_init_time and expiration_date), droped correlating columns.

**1% of Data is used in the kernel.**

In [ ]:
# Load data
df = pd.read_csv('../input/train.csv')

# 1% sample of items
df = df.sample(frac=0.01)

In [ ]:
# Load and join songs data
songs = pd.read_csv('../input/songs.csv')
df = pd.merge(df, songs, on='song_id', how='left')
del songs

# Load and join songs data
members = pd.read_csv('../input/members.csv')
df = pd.merge(df, members, on='msno', how='left')
del members

In [ ]:
df.info()

In [ ]:
# Count Na in %
df.isnull().sum()/df.isnull().count()*100

In [ ]:
# Replace NA
for i in df.select_dtypes(include=['object']).columns:
    df[i][df[i].isnull()] = 'unknown'
df = df.fillna(value=0)

In [ ]:
# Create Dates

# registration_init_time
df.registration_init_time = pd.to_datetime(df.registration_init_time, format='%Y%m%d', errors='ignore')
df['registration_init_time_year'] = df['registration_init_time'].dt.year
df['registration_init_time_month'] = df['registration_init_time'].dt.month
df['registration_init_time_day'] = df['registration_init_time'].dt.day

# expiration_date
df.expiration_date = pd.to_datetime(df.expiration_date,  format='%Y%m%d', errors='ignore')
df['expiration_date_year'] = df['expiration_date'].dt.year
df['expiration_date_month'] = df['expiration_date'].dt.month
df['expiration_date_day'] = df['expiration_date'].dt.day

In [ ]:
df.head()

In [ ]:
# Dates to categoty
df['registration_init_time'] = df['registration_init_time'].astype('category')
df['expiration_date'] = df['expiration_date'].astype('category')

In [ ]:
# Object data to category
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')
    
# Encoding categorical features
for col in df.select_dtypes(include=['category']).columns:
    df[col] = df[col].cat.codes

In [ ]:
# Сorrelation matrix
plt.figure(figsize=[7,5])
sns.heatmap(df.corr())
plt.show()

In [ ]:
# Drop columns
df = df.drop(['expiration_date', 'lyricist'], 1)

## Random Forest

I selected the most informative columns using Random Forest Model.

In [ ]:
# Model with the best estimator
model = ensemble.RandomForestClassifier(n_estimators=250, max_depth=25)
model.fit(df[df.columns[df.columns != 'target']], df.target)

In [ ]:
df_plot = pd.DataFrame({'features': df.columns[df.columns != 'target'],
                        'importances': model.feature_importances_})
df_plot = df_plot.sort_values('importances', ascending=False)

In [ ]:
plt.figure(figsize=[11,5])
sns.barplot(x = df_plot.importances, y = df_plot.features)
plt.title('Importances of Features Plot')
plt.show()

In [ ]:
# Drop columns with importances < 0.04
df = df.drop(df_plot.features[df_plot.importances < 0.04].tolist(), 1)

In [ ]:
# Selected columns
df.columns

## XGboost

In [ ]:
# Train & Test split
target = df.pop('target')
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(df, target, test_size = 0.3)

# Delete df
del df

The best estimator was selected by Randomized Grid Search with Stratified Shuffle Split Cross Validation.

\# Create model

model = xgb.XGBClassifier()

\# Create parameters grid

parameters_grid = {
    'learning_rate' : [0.1, 0.2, 0.5],
    'max_depth' : [5, 10, 15],
    'n_estimators' : [150, 250, 300],
    'min_child_weight' : [3, 5, 10]
}

\# Strategy of cross validation

cv = cross_validation.StratifiedShuffleSplit(train_labels, n_iter = 3, test_size = 0.3)

\# Create Grid search Strategy

grid_cv = grid_search.RandomizedSearchCV(model, parameters_grid, scoring = 'accuracy', cv = cv)

\# Fit model by Grid

grid_cv.fit(train_data, train_labels)

\# Model with the best estimator 

grid_cv.best_estimator_

In [ ]:
# Create model
model = xgb.XGBClassifier(learning_rate=0.1, max_depth=15, min_child_weight=5, n_estimators=250)
model.fit(train_data, train_labels)

In [ ]:
# Predicting
predict_labels = model.predict(test_data)

## The results of the algorithm

In [ ]:
print(metrics.classification_report(test_labels, predict_labels))